# MELODI Lite API Example Usage

In [2]:
import json
import operator
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from environs import Env

## Configure parameters

In [7]:
API_URL = "http://melodi-lite.mrcieu.ac.uk/api/"

requests.get(f"{API_URL}/status").json()

True

### Get enriched SemMedDB objects for a given search term

Starting from a PubMed search, return all SemMedDB subject-predicate-object triples, their enrcichment metrics and publication information 

In [ ]:
query_term='PCSK9'
